### Link to training notebook:

[https://www.kaggle.com/code/dlaststark/feedback-ell-roberta-large-train](http://)

## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

from transformers import AutoTokenizer, TFAutoModel

tqdm.pandas()
np.random.seed(2022)
tf.random.set_seed(2022)

## Hyperparameters

In [2]:
! cp -r ../input/feedback-ell-roberta-large-train/roberta-large-tokenizer ./
! mv ./roberta-large-tokenizer/tokenizer_config.json ./roberta-large-tokenizer/config.json

In [3]:
class Config:
    
    FOLDS = 5
    VERBOSE = 1
    MAX_LEN = 512
    BATCH_SIZE = 32
    MODEL_PATH = '../input/feedback-ell-roberta-large-train/roberta-large-model'
    TOKENIZER_PATH = './roberta-large-tokenizer'

config = Config()

## Load test dataset

In [4]:
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
print(f"test: {test.shape}")
test.head()

test: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


## Build the model

In [5]:
def encode_text(text, tokenizer):
    
    encoded = tokenizer.batch_encode_plus(
        text,
        add_special_tokens=True,
        max_length=config.MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="tf",
    )

    input_ids = np.array(encoded["input_ids"], dtype="int32")
    attention_masks = np.array(encoded["attention_mask"], dtype="int32")

    return {
        "input_ids": input_ids,
        "attention_masks": attention_masks
    }

In [6]:
def feedback_model(transformer_model):
    
    input_ids = layers.Input(shape=(config.MAX_LEN,), dtype=tf.int32, name="input_ids")
    attention_mask = layers.Input(shape=(config.MAX_LEN,), dtype=tf.int32, name="attention_mask")

    bert_model = transformer_model(input_ids, attention_mask=attention_mask)
    
    last_hidden_state, pooler_output = bert_model[0], bert_model[1]
    
    x = layers.Concatenate()([
        pooler_output,
        layers.GlobalAveragePooling1D()(last_hidden_state),
        layers.GlobalMaxPooling1D()(last_hidden_state)
    ])
    x = layers.Dropout(rate=0.35)(x)
    
    transformer_output = layers.Dense(units=1024, activation='gelu')(x)
    
    re = layers.Reshape((8, 8, 16))(transformer_output)
    
    x1 = layers.Conv2D(filters=32, kernel_size=3, strides=1,  
                       padding='same', activation='gelu')(re)
    x2 = layers.SpatialDropout2D(rate=0.2)(x1)
    
    x2 = layers.Conv2D(filters=48, kernel_size=3, strides=1, 
                       padding='same', activation='gelu')(x2)
    x3 = layers.SpatialDropout2D(rate=0.2)(x2)
    
    x3 = layers.Conv2D(filters=64, kernel_size=3, strides=1, 
                       padding='same', activation='gelu')(x3)
    
    x4 = layers.MaxPool2D()(x3)
    x4 = layers.Flatten()(x4)
    
    add0 = layers.Add()([transformer_output, x4])
    
    x_output = layers.Average()([
        layers.Dense(units=6, activation='linear')(layers.Dropout(rate=0.35)(add0)),
        layers.Dense(units=6, activation='linear')(layers.Dropout(rate=0.25)(add0)),
        layers.Dense(units=6, activation='linear')(layers.Dropout(rate=0.15)(add0))
    ])

    model = Model(inputs=[input_ids, attention_mask], 
                  outputs=x_output, 
                  name='Feedback_TFRoberta_Large_Model')
    return model

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.TOKENIZER_PATH)
transformer_model = TFAutoModel.from_pretrained(config.MODEL_PATH)

test_data = encode_text(test['full_text'].tolist(), tokenizer)

2022-08-31 13:57:41.422227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 13:57:41.423362: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 13:57:41.424065: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 13:57:41.424978: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

## Make predictions

In [8]:
test_preds = []

for idx in range(config.FOLDS):
    print(f">>>>>  Fold-{idx+1}  <<<<<")
    
    model = feedback_model(transformer_model)
    model.load_weights(f"../input/feedback-ell-roberta-large-train/Feedback_TFRoberta_Large_Model_{idx+1}C.h5")

    y_pred = model.predict(
        (np.asarray(test_data['input_ids']),
         np.asarray(test_data['attention_masks'])), 
        batch_size=config.BATCH_SIZE, 
        verbose=config.VERBOSE
    )
    
    test_preds.append(y_pred)
    
    del model, y_pred
    gc.collect()
    
    print()

>>>>>  Fold-1  <<<<<


2022-08-31 13:58:23.545430: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-31 13:58:29.843715: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1/1 [==============================] - 11s 11s/step

>>>>>  Fold-2  <<<<<
1/1 [==============================] - 5s 5s/step

>>>>>  Fold-3  <<<<<
1/1 [==============================] - 5s 5s/step

>>>>>  Fold-4  <<<<<
1/1 [==============================] - 5s 5s/step

>>>>>  Fold-5  <<<<<
1/1 [==============================] - 5s 5s/step



In [9]:
y_pred_final = np.mean(test_preds, axis=0)

## Create submission file

In [10]:
submission = pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv")

submission['cohesion'] = y_pred_final[:,0]
submission['syntax'] = y_pred_final[:,1]
submission['vocabulary'] = y_pred_final[:,2]
submission['phraseology'] = y_pred_final[:,3]
submission['grammar'] = y_pred_final[:,4]
submission['conventions'] = y_pred_final[:,5]

submission.to_csv("./submission.csv", index=False)
submission.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.624945,2.766888,2.795981,2.745884,2.631725,2.698045
1,000BAD50D026,2.354248,2.435251,2.487714,2.383238,2.209692,2.406398
2,00367BB2546B,3.432826,3.585508,3.507679,3.544212,3.450433,3.516163


In [11]:
## Good Day!!